In [ ]:
!pip install ultralytics opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
import os
import shutil
import json
from ultralytics import YOLO

In [ ]:
import torch
use_cuda = True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The code below was not publicly available and required special conversions from our file format of JSON to the appropriate YAML file. In this case, seeing as this was for the processing stage, we did utilize ChatGPT to help us write some snippets to convert the files accordingly, with some edits on our end to make sure the code worked properly. We fully understand how the code works, as it essentially takes the JSON file, converts the bounding box dimension in the JSON file to a text file suitable for YOLOv8 interpretation, and creates a YAML file that leads YOLOv8 to find the appropriate images and txt files for processing.

In [ ]:
def convert_json_to_yolo(json_path, img_w, img_h, label_output_path, class_map={"pothole": 0}):
    with open(json_path, 'r') as f:
        data = json.load(f)

    lines = []
    for obj in data["objects"]:
        cls_id = class_map[obj["classTitle"]]
        (x1, y1), (x2, y2) = obj["points"]["exterior"]

        x_center = (x1 + x2) / 2.0 / img_w
        y_center = (y1 + y2) / 2.0 / img_h
        width = abs(x2 - x1) / img_w
        height = abs(y2 - y1) / img_h

        lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    with open(label_output_path, "w") as f:
        f.write("\n".join(lines))

In [ ]:
def process_split(split):
    img_dir = os.path.join(root, split, 'img')
    json_dir = os.path.join(root, split, 'json')

    image_files = [f for f in os.listdir(img_dir)]

    for file in image_files:
        base = os.path.splitext(file)[0]
        img_path = os.path.join(img_dir, file)
        json_path = os.path.join(json_dir, file + ".json")
        dest_img_path = f"{root}/images/{split}/{file}"
        dest_label_path = f"{root}/labels/{split}/{base}.txt"

        shutil.copy(img_path, dest_img_path)

        img_dim = 512

        if os.path.exists(json_path):
            convert_json_to_yolo(json_path, img_dim, img_dim, dest_label_path)
        else:
            open(dest_label_path, "w").close()

In [ ]:
root = '/content/drive/MyDrive/APS360_Project/RDD_2022_ProcessedData/'
splits = ['train', 'val', 'test']

for split in splits:
    os.makedirs(f'{root}/images/{split}', exist_ok=True)
    os.makedirs(f'{root}/labels/{split}', exist_ok=True)

os.makedirs(f'{root}/data.yaml', exist_ok=True)
os.makedirs(f'{root}/data_test.yaml', exist_ok=True)

In [ ]:
process_split('train')

In [ ]:
process_split('val')

In [ ]:
process_split('test')

In [ ]:
with open(f"{root}/data.yaml", "w") as f:
    f.write(
        """
train: /content/drive/MyDrive/APS360_Project/RDD_2022_ProcessedData/images/train
val: /content/drive/MyDrive/APS360_Project/RDD_2022_ProcessedData/images/val

nc: 1
names: ['pothole']
        """.strip()
    )

In [ ]:
model = YOLO("yolov8s.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 21.5M/21.5M [00:00<00:00, 242MB/s]


In [ ]:
model.train(data=f"{root}/data.yaml", epochs=20, imgsz=512, batch=128, freeze=10)

Ultralytics 8.3.165 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=128, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/APS360_Project/RDD_2022_ProcessedData//data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspec

100%|██████████| 755k/755k [00:00<00:00, 22.3MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

 22        [15, 18, 21]  1   2116435  ultralytics.nn.modules.head.Detect           [1, [128, 256, 512]]          
Model summary: 129 layers, 11,135,987 parameters, 11,135,971 gradients, 28.6 GFLOPs

Transferred 349/355 items from pretrained weights
Freezing layer 'model.0.conv.weight'
Freezing layer 'model.0.bn.weight'
Freezing layer 'model.0.bn.bias'
Freezing layer 'model.1.conv.weight'
Freezing layer 'model.1.bn.weight'
Freezing layer 'model.1.bn.bias'
Freezing layer 'model.2.cv1.conv.weight'
Freezing layer 'model.2.cv1.bn.weight'
Freezing layer 'model.2.cv1.bn.bias'
Freezing layer 'model.2.cv2.conv.weight'
Freezing layer 'model.2.cv2.bn.weight'
Freezing layer 'model.2.cv2.bn.bias'
Freezing layer 'model.2.m.0.cv1.conv.weight'
Freezing layer 'model.2.m.0.cv1.bn.weight'
Freezing layer 'model.2.m.0.cv1.bn.bias'
Freezing layer 'model.2.m.0.cv2.conv.weight'
Freezing layer 'model.2.m.0.cv2.bn.weight'
Freezing layer 'model.2.m.0.cv2.bn.bias'
Freezing layer 'model.3.conv.weight'
Freezing lay

100%|██████████| 5.35M/5.35M [00:00<00:00, 88.0MB/s]


...

                   all        658        340      0.279      0.341      0.207     0.0786



20 epochs completed in 0.495 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 22.5MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.165 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:10<00:00,  3.50s/it]


                   all        658        340      0.239        0.4      0.206     0.0792
Speed: 0.1ms preprocess, 3.2ms inference, 0.0ms loss, 7.7ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7972f2b6c190>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
with open(f"{root}/data_test.yaml", "w") as f:
    f.write(
        """
train: /content/drive/MyDrive/APS360_Project/RDD_2022_ProcessedData/images/train
val: /content/drive/MyDrive/APS360_Project/RDD_2022_ProcessedData/images/val
test: /content/drive/MyDrive/APS360_Project/RDD_2022_ProcessedData/images/test

nc: 1
names: ['pothole']
        """.strip()
    )

In [ ]:
model.val(data=f'{root}data_test.yaml', split='test', imgsz=512, save=True)

Ultralytics 8.3.165 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.6±0.2 ms, read: 42.1±13.3 MB/s, size: 108.8 KB)


val: Scanning /content/drive/.shortcut-targets-by-id/1cifpQDRBcJgUxkcQMi6UsUSFPn7RfTNX/APS360_Project/RDD_2022_ProcessedData/labels/test... 661 images, 441 backgrounds, 0 corrupt: 100%|██████████| 661/661 [00:08<00:00, 76.73it/s] 


val: New cache created: /content/drive/.shortcut-targets-by-id/1cifpQDRBcJgUxkcQMi6UsUSFPn7RfTNX/APS360_Project/RDD_2022_ProcessedData/labels/test.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:14<00:00,  2.46s/it]


                   all        661        394      0.332      0.297      0.202     0.0763
Speed: 5.8ms preprocess, 7.0ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/train27


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7972f0694110>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 